<a href="https://colab.research.google.com/github/farhanhubble/google-colab/blob/master/advanced_lane_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [0]:
!ls /content/drive/My\ Drive/data_driver_view